<a href="https://colab.research.google.com/github/arjunkoneru/QA-for-lectures/blob/main/Demo_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Demo for TAILOR

Login with mrp2taillor@gmail.com(ask shashank for password if you don't have) to import fine-tuned or non fine-tuned transcripts


In [ ]:
#@title Load Libraries, BERT QA model and Helper Functions
import json
import pandas as pd
import re
import string
import numpy as np
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq
from sklearn.metrics.pairwise import cosine_similarity
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import nltk
from nltk.stem.porter import *
stemmer = PorterStemmer()
stemmer.stem('mean square')
import re
regex = re.compile('[^a-zA-Z]')
qa_model = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(qa_model)
model = AutoModelForQuestionAnswering.from_pretrained(qa_model)
def remove_punctuation(txt):
  txt_nopunct = "".join([c for c in txt if c not in string.punctuation]) ## Remove punctuation
  txt_nopunct = txt_nopunct.lower() ## Lower-case conversion
  return txt_nopunct

def iterativePadding(doc_id,nopunct_sentences,query,max_length):
  i = 0
  r = doc_id+1
  l = doc_id-1
  r_end = False
  l_end = False
  passage = nopunct_sentences[doc_id]
  new_passage = passage
  while IsInputValid(query, new_passage,max_length)[0] and not(r_end and l_end):
    # print(len(new_passage), IsInputValid(query, new_passage)[0])
    passage = new_passage
    if i%2 == 0:
      if r < len(nopunct_sentences):
        new_passage += ".\n" + nopunct_sentences[r]
        r+=1
      else: # Unnecessary unless the article is too short to include all of it in one paragraph
        r_end = True
    else:
      if l >= 0:
        new_passage = nopunct_sentences[l] +  ".\n" + new_passage
        l-=1
      else:
        l_end = True
    i += 1
  
  return passage

def IsInputValid(question,paragraph,max_length):
  global tokenizer
  start_time = time.time()
  inputs = tokenizer.encode_plus(question, paragraph, add_special_tokens=True, return_tensors="pt",max_length= 512)
  input_ids = inputs["input_ids"].tolist()[0]
  text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  elapsed_time = time.time() - start_time
  if len(text_tokens) < max_length:
    return True, inputs,input_ids
  else:
    return False, inputs,input_ids

def VanillaTFIDF(nopunct_sentences,question,max_rank):
  vectorizer = TfidfVectorizer()
  weights = vectorizer.fit_transform(nopunct_sentences)
  weight_matrix = pd.DataFrame(weights.todense(), columns= vectorizer.get_feature_names())
  tokenized_query = remove_punctuation(question).split(' ')
  total_documents, vocab_size = weight_matrix.shape
  zero_scores = np.zeros((total_documents,))
  final_scores = pd.Series(zero_scores)
  for term in tokenized_query:
    if term in vectorizer.get_feature_names():
      term_weights = weight_matrix[term]
      final_scores+=term_weights
  nonzero_relevance_scores = final_scores[final_scores!=0].sort_values(ascending = False)
  doc_ids = nonzero_relevance_scores[0:max_rank].index
  relevant_docs = [nopunct_sentences[int(i)] for i in doc_ids]
  return doc_ids,relevant_docs

def SimilarityTFIDF(nopunct_sentences,question,max_rank):
  docvector_vectorizer = TfidfVectorizer()
  docvector_weights = docvector_vectorizer.fit_transform(nopunct_sentences)
  features = docvector_vectorizer.get_feature_names()
  tokenized_query = remove_punctuation(question).split(' ')
  idf = docvector_vectorizer._tfidf.idf_
  feature_weights = dict(zip(features, idf))
  question_weights = []
  for feature in features:
    if feature in tokenized_query:
      question_weights.append(feature_weights[feature])
    else:
      question_weights.append(0)
  question_weights = np.asarray(question_weights)
  doc_matrix = cosine_similarity(question_weights.reshape(1,-1), docvector_weights)
  doc_matrix = doc_matrix.reshape((doc_matrix.shape[1],))
  doc_ids = heapq.nlargest(max_rank, range(len(doc_matrix)), doc_matrix.__getitem__)
  topsents = [nopunct_sentences[i] for i in doc_ids]
  return doc_ids,topsents

def RunQA(inputs,input_ids,model):
  global tokenizer
  answer_start_scores, answer_end_scores = model(**inputs)
  answer_start = torch.argmax(
      answer_start_scores
  )  # Get the most likely beginning of answer with the argmax of the score
  score = torch.max(answer_start_scores).item()
  answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
  return answer,score


     |████████████████████████████████| 675kB 4.4MB/s 
     |████████████████████████████████| 1.1MB 23.1MB/s 
     |████████████████████████████████| 3.8MB 34.0MB/s 
     |████████████████████████████████| 890kB 49.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=657e4f9c2aaf6bd9b02f67bf604708bb15d7bd482b31ee0ec3ef2994458dbf5d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#@title Import Transcripts from Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#@title IR Settings(Don't Change if not needed)
IR_similarity = True #@param {type:"boolean"}
max_passage_length = 511 #@param {type:"slider", min:0, max:511, step:1}
padding_approach =  1#@param {type:"number"}
max_rank = 10

In [ ]:
#@title Enter Lecture name and question here
lecture_name = "Introduction to Deep Learning" #@param {type:"string"}
question = "Can you give an example where neural network can be applied?" #@param {type:"string"}


In [ ]:
#@title Read Transcript
transcripts_path = '/content/drive/My Drive/Tailor/ASR/no_finetuning /punctuated_transcripts/'
#lecture_path = transcripts_path + lecture_name + '.txt'
lecture_path = '/content/drive/My Drive/Tailor/ASR/ds_0.6.1_inference/mit_introduction_to_deeplearning_transcription_punctuated/MIT_DL.txt'

transcript_file = open(lecture_path,'r')
transcript = transcript_file.read()

In [ ]:
#@title Output Passage and BERT's Answer
transcript_stemmed = stemmer.stem(transcript)
question = regex.sub(' ', question)
question = stemmer.stem(question)
sentences = transcript_stemmed.replace('\n',' ').split(". ")
nopunct_sentences =  list(map(remove_punctuation, sentences)) 
nopunct_sentences = list(filter(None, nopunct_sentences))  ## Remove empty items occured by splitting
if not IR_similarity:
  doc_ids,relevant_docs = VanillaTFIDF(nopunct_sentences,question,max_rank)
else:
  doc_ids,relevant_docs = SimilarityTFIDF(nopunct_sentences,question,max_rank)
try:
  sentences = transcript.replace('\n',' ').split(". ")
  nopunct_sentences =  list(map(remove_punctuation, sentences)) 
  nopunct_sentences = list(filter(None, nopunct_sentences)) 
  if padding_approach == 1:
    passage = iterativePadding(doc_ids[0],nopunct_sentences,question,max_passage_length)
  elif padding_approach == 2:
    first_passage = iterativePadding(doc_ids[0],nopunct_sentences,question,max_passage_length/2)
    second_passage = iterativePadding(doc_ids[1],nopunct_sentences,question,max_passage_length/2)
    passage = first_passage + ". " + second_passage
  elif padding_approach == 0:
    passage = ". ".join(relevant_docs)
except:
  print("IR unable to find sentences related to question")
print('##############')
print('Question:\n')
print(question)
print("##############")
print("IR PASSAGE:\n")
print(passage)
valid,inputs,input_ids = IsInputValid(question,passage,max_passage_length)
answer,score = RunQA(inputs,input_ids,model)
answer = answer.replace(' . ',' ')
print("#############")
print("Predicted Answer: "+answer)

##############
Question:

can you give an example where neural network can be applied 
##############
IR PASSAGE:

how strongly or aggressively do you want to  step towards that gradient.
in code the picture looks very similar.
so to  implement gradient descent is just a few lines of code just like the pseudocode.
 you can initialize your weights randomly in the first line.
you can compute your  loss with respect to those gradients and with respect to those predictions and  your data.
given that gradient you just update your weights in the opposite  direction of that of that vector right.
 now the magic line here is actually how do you compute that gradient and thats  something i havent told you and thats something its not easy at all.
so the  question is given a loss and given all of our weights in our network how do we  know which way is good which way is a good place to move given all of this  information.
and i never told you about that but thats a process called back  propagation.